# 数据集创建&处理

In [2]:
# from modelarts.session import Session
# session = Session()
# session.obs.download_file(src_obs_file="obs://llddy/LUKE_mindspore/data/dev_data.npy", dst_local_dir="./dev_data.npy")

In [1]:
from dataset.build_dataset import build_dataset
import mindspore.dataset as ds
import os
import numpy as np
from mindspore.mindrecord import FileWriter
import json

In [2]:
#创建mindrecord

FEATURES_FILE = "./data/dev_data.npy"
#FEATURES_FILE = "./data/json_features.npy"
features = np.load(FEATURES_FILE)
list_dict = []
for item in features:
    dict_temp = json.loads(item)
    list_dict.append(dict_temp)
SQUAD_MINDRECORD_FILE = "./data/dev_features.mindrecord"

In [3]:
pad = lambda a,i : a[0:i] if len(a) > i else a + [0] * (i-len(a))
pad_entity = lambda a,i : a[0:i] if len(a) > i else np.append(a,[-1] * (i-len(a)))

for slist in list_dict:
    slist["entity_attention_mask"] = pad(slist["entity_attention_mask"], 24)
    slist["entity_ids"] = pad(slist["entity_attention_mask"], 24)
    slist["entity_segment_ids"] = pad(slist["entity_segment_ids"], 24)
    
    slist["word_ids"] = pad(slist["word_ids"], 256)
    slist["word_segment_ids"] = pad(slist["word_segment_ids"], 256)
    slist["word_attention_mask"] = pad(slist["word_attention_mask"], 256)
    slist["entity_position_ids"] = np.array(slist["entity_position_ids"]).flatten()
    slist["entity_position_ids"] = pad_entity(slist["entity_position_ids"], 256)


if os.path.exists(SQUAD_MINDRECORD_FILE):
    os.remove(SQUAD_MINDRECORD_FILE)
    os.remove(SQUAD_MINDRECORD_FILE + ".db")

writer = FileWriter(file_name=SQUAD_MINDRECORD_FILE, shard_num=1)

data_schema = {
    "unique_id": {"type": "int32", "shape": [-1]},
    "word_ids": {"type": "int32", "shape": [-1]},
    "word_segment_ids": {"type": "int32", "shape": [-1]},
    "word_attention_mask": {"type": "int32", "shape": [-1]},
    "entity_ids": {"type": "int32", "shape": [-1]},
    "entity_position_ids": {"type": "int32", "shape": [-1]},
    "entity_segment_ids": {"type": "int32", "shape": [-1]},
    "entity_attention_mask": {"type": "int32", "shape": [-1]},
    #"start_positions": {"type": "int32", "shape": [-1]},
    #"end_positions": {"type": "int32", "shape": [-1]}
}
writer.add_schema(data_schema, "it is a preprocessed squad dataset")

data = []
i = 0
for item in list_dict:
    i += 1
    sample = {
        "unique_id": np.array(item["unique_id"], dtype=np.int32),
        "word_ids": np.array(item["word_ids"], dtype=np.int32),
        "word_segment_ids": np.array(item["word_segment_ids"], dtype=np.int32),
        "word_attention_mask": np.array(item["word_attention_mask"], dtype=np.int32),
        "entity_ids": np.array(item["entity_ids"], dtype=np.int32),
        "entity_position_ids": np.array(item["entity_position_ids"], dtype=np.int32),
        "entity_segment_ids": np.array(item["entity_segment_ids"], dtype=np.int32),
        "entity_attention_mask": np.array(item["entity_attention_mask"], dtype=np.int32),
        #"start_positions": np.array(item["start_positions"], dtype=np.int32),
        #"end_positions": np.array(item["end_positions"], dtype=np.int32),
    }

    data.append(sample)
    #print(sample)
    if i % 10 == 0:
        writer.write_raw_data(data)
        data = []

if data:
    writer.write_raw_data(data)

writer.commit()

MSRStatus.SUCCESS

In [4]:
#SQUAD_MINDRECORD_FILE = "./data/dev_features.mindrecord"
data_set = ds.MindDataset(dataset_file=SQUAD_MINDRECORD_FILE)
# count = 0
# for item in data_set.create_dict_iterator():
#     #print(item)
#     count += 1
# print("Got {} samples".format(count))

In [5]:
data_set = data_set.batch(24)
data_sample = next(data_set.create_dict_iterator())
data_sample

{'entity_attention_mask': Tensor(shape=[24, 24], dtype=Int32, value=
 [[0, 0, 0 ... 0, 0, 0],
  [0, 0, 0 ... 0, 0, 0],
  [1, 0, 0 ... 0, 0, 0],
  ...
  [1, 1, 0 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0]]),
 'entity_ids': Tensor(shape=[24, 24], dtype=Int32, value=
 [[0, 0, 0 ... 0, 0, 0],
  [0, 0, 0 ... 0, 0, 0],
  [1, 0, 0 ... 0, 0, 0],
  ...
  [1, 1, 0 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0]]),
 'entity_position_ids': Tensor(shape=[24, 256], dtype=Int32, value=
 [[-1, -1, -1 ... -1, -1, -1],
  [-1, -1, -1 ... -1, -1, -1],
  [11, 12, -1 ... -1, -1, -1],
  ...
  [29, 30, 31 ... -1, -1, -1],
  [11, 12, -1 ... -1, -1, -1],
  [77, -1, -1 ... -1, -1, -1]]),
 'entity_segment_ids': Tensor(shape=[24, 24], dtype=Int32, value=
 [[0, 0, 0 ... 0, 0, 0],
  [0, 0, 0 ... 0, 0, 0],
  [0, 0, 0 ... 0, 0, 0],
  ...
  [0, 0, 0 ... 0, 0, 0],
  [0, 0, 0 ... 0, 0, 0],
  [0, 0, 0 ... 0, 0, 0]]),
 'unique_id': Tensor(shape=[24, 1], dtype=Int32, value=
 [[1000005274],

In [6]:
word_ids = data_sample["word_ids"]
word_segment_ids= data_sample["word_segment_ids"]
word_attention_mask= data_sample["word_attention_mask"]
entity_ids= data_sample["entity_ids"]
entity_position_ids= data_sample["entity_position_ids"]
entity_segment_ids= data_sample["entity_segment_ids"]
entity_attention_mask= data_sample["entity_attention_mask"]

# Model

In [7]:
from readingcomprehension.models.luke import LukeForReadingComprehension, LukeEntityAwareAttentionModel
import mindspore.common.dtype as mstype
from model.bert_model import BertConfig
from mindspore import context
from model.luke import LukeModel, EntityAwareEncoder
from mindspore import Tensor, context
from mindspore import dtype as mstype
import mindspore.ops as ops
import mindspore.nn as nn
from model.bert_model import BertOutput
from mindspore.common.initializer import TruncatedNormal
from mindspore.ops import composite as C
import mindspore
from mindspore.ops import operations as P
from mindspore.train.model import Model
context.set_context(mode=context.GRAPH_MODE, device_target="Ascend")

## 简单测试

In [9]:
config = BertConfig()
Luke_model = LukeForReadingComprehension(config)
model = Model(Luke_model)

In [10]:
logits = model.predict(word_ids,word_segment_ids,word_attention_mask,entity_ids,entity_position_ids,entity_segment_ids,entity_attention_mask)

In [11]:
logits

(Tensor(shape=[24, 256], dtype=Float32, value=
 [[ 2.92592585e-01,  2.70484447e-01,  2.65146255e-01 ...  2.63161600e-01,  2.59273797e-01,  2.95242786e-01],
  [ 3.18681479e-01,  2.97804475e-01,  2.70562023e-01 ...  2.99310863e-01,  2.90683925e-01,  3.07460636e-01],
  [ 2.56083131e-01,  2.35625535e-01,  2.13921010e-01 ...  2.38565832e-01,  2.26389676e-01,  2.43567199e-01],
  ...
  [ 2.66339034e-01,  2.45516524e-01,  2.45532036e-01 ...  2.48828977e-01,  2.38703132e-01,  2.56241202e-01],
  [ 3.29715371e-01,  3.08946311e-01,  3.06750625e-01 ...  3.09931695e-01,  3.09648693e-01,  2.97910243e-01],
  [ 2.54786551e-01,  2.23702818e-01,  2.21459359e-01 ...  2.36167192e-01,  2.27386415e-01,  2.43492305e-01]]),
 Tensor(shape=[24, 256], dtype=Float32, value=
 [[-4.45168614e-01, -4.17687595e-01, -4.18160260e-01 ... -4.17240530e-01, -4.13319558e-01, -4.30681825e-01],
  [-4.23104525e-01, -3.96651030e-01, -3.95476073e-01 ... -3.96843135e-01, -3.93367976e-01, -4.17906284e-01],
  [-4.35993910e-01, -4.097

# do_eval

In [8]:
!pip install tqdm
from tqdm import tqdm

Looking in indexes: http://100.125.0.87:32021/repository/pypi/simple


In [9]:
from mindspore.train.serialization import load_checkpoint, load_param_into_net
from mindspore.train.model import Model
import collections
def do_eval(dataset=None, load_checkpoint_path="", eval_batch_size=1):
    config = BertConfig()
    Luke_model = LukeForReadingComprehension(config)
    Luke_model.set_train(False)
    model = Model(Luke_model)
    param_dict = load_checkpoint(load_checkpoint_path)
    load_param_into_net(Luke_model, param_dict)
    output = []
    #model = Model(Luke_model)
    
    RawResult = collections.namedtuple("RawResult", ["unique_id", "start_logits", "end_logits"])
    columns_list = ["unique_id", "word_ids", "word_segment_ids", "word_attention_mask", "entity_ids", "entity_position_ids", "entity_segment_ids", "entity_attention_mask"]
    data_set = ds.MindDataset(dataset_file=SQUAD_MINDRECORD_FILE)
    data_set = data_set.batch(eval_batch_size,drop_remainder=True)
    for data in tqdm(data_set.create_dict_iterator(num_epochs=1)):
        input_data = []
        for i in columns_list:
            input_data.append(data[i])

        unique_id, word_ids,word_segment_ids,word_attention_mask,entity_ids,entity_position_ids,entity_segment_ids,entity_attention_mask = input_data
        #print(unique_id)
        logits = model.predict(word_ids,word_segment_ids,word_attention_mask,entity_ids,entity_position_ids,entity_segment_ids,entity_attention_mask)
        ids = unique_id.asnumpy()
        start = logits[0].asnumpy()
        end = logits[1].asnumpy()

        for i in range(eval_batch_size):
            unique_id = int(ids[i])
            start_logits = [float(x) for x in start[i].flat]
            end_logits = [float(x) for x in end[i].flat]
            output.append(RawResult(
                unique_id=unique_id,
                start_logits=start_logits,
                end_logits=end_logits))
        #print(word_ids.shape)
    return output

In [10]:
dev_dataset = ds.MindDataset(dataset_file=SQUAD_MINDRECORD_FILE)

In [11]:
dev_dataset.get_dataset_size()

10779

In [ ]:
outputs = do_eval(dev_dataset, load_checkpoint_path = "./luke-large-qa.ckpt", eval_batch_size = 24)

67it [21:43, 18.29s/it]

In [ ]:
outputs

In [ ]:
import numpy as np
np.save('dev_outputs.npy',outputs)

# 验证

In [ ]:
eval_features = np.load("./data/dev_data.npy")
eval_examples = read_squad_examples(args_opt.eval_json_path, False)
all_predictions = write_predictions(eval_examples, eval_features, outputs, 20, 30, True)
SQuad_postprocess(args_opt.eval_json_path, all_predictions, output_metrics="output.json")